In [1]:
import os
import sys 
import random
import pandas as pd
import numpy as np

import skimage.io
import skimage.measure
import matplotlib.pyplot as plt
import requests
import json
from tqdm import tqdm

from PIL import Image
import glob 

import pydicom 
from pprint import pprint 
import requests

%matplotlib inline

In [2]:
# !pip install --upgrade mdai 

MODULE_DIR = os.path.abspath('../mdai-client-py')
sys.path.append(MODULE_DIR)

import mdai
import importlib
importlib.reload(mdai)

mdai.__version__

'0.0.5'

In [3]:
ROOT_DIR = os.path.abspath('')
DICOM_DIR = os.path.join(DATA_ROOT_DIR, 'dicom')

### Create `mdai` client

In [ ]:
mdai_client = mdai.Client(domain='public.md.ai', access_token="ACCESS_TOKEN")
p = mdai_client.project('PROJECT_ID', path='data')

In [ ]:
# get labed_id 
p.show_label_groups()

In [ ]:
# get dataset id 
p.show_datasets() 

## User need to specify 
- dataset id, e.g., 'D_qGQdpN'
- label id (to clone from), e.g., 'L_egJRyg'

In [7]:
# specify label ids
label_ids = ['L_38Y7Jl', 'L_z8xEkB']

In [8]:
# specify dataset id to upload annotations to
dataset_id = ['D_8ogmzN']

### Prepare annotations 

In [5]:
# read annotations 
anns_csv = pd.read_csv(os.path.join(ROOT_DIR, 'annotaions.csv'))

In [6]:
anns_csv.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [14]:
def create_annotations(anns): 
    
    # label id (or type)
    # dataset id 
    # StudyInstanceUID or SeriesInstanceUID or SOPInstance UID 
    # data (only if local AND SOPInstance UID ?) 
    
    annotations = []

    for index, row in tqdm(anns.iterrows()):
        
        # build annotation 
        
        a = {
            'labelId': label_id, # or label_type ?
            'dicomDatasetId': dicom_dataset_id, # dataset id 
            'SOPInstanceUID': sop_instance_uid, 
            'SeriesInstanceUID': series_instance_uid, 
            'StudyInstanceUID' : study_instance_uid, 
            'data':json.dumps({data}), # could be null  
        }
        annotations.append(a)
    
    return annotations 

In [15]:
anns_train = create_annotations(train_csv, 'train') 

28989it [00:29, 979.11it/s]


In [16]:
anns_df = pd.DataFrame(anns_train)

In [18]:
anns_df.head(20)

,SOPInstanceUID,data,dicomDatasetId,labelId
0,1.2.276.0.7230010.3.1.4.8323329.28530.15178744...,NaN,78,457
1,1.2.276.0.7230010.3.1.4.8323329.26024.15178744...,NaN,78,457
2,1.2.276.0.7230010.3.1.4.8323329.11252.15178743...,NaN,78,457
3,1.2.276.0.7230010.3.1.4.8323329.2293.151787429...,NaN,78,457
4,1.2.276.0.7230010.3.1.4.8323329.6379.151787432...,"{""x"": 264.0, ""y"": 152.0, ""width"": 213.0, ""heig...",78,458
5,1.2.276.0.7230010.3.1.4.8323329.6379.151787432...,"{""x"": 562.0, ""y"": 152.0, ""width"": 256.0, ""heig...",78,458
6,1.2.276.0.7230010.3.1.4.8323329.18028.15178744...,NaN,78,457
7,1.2.276.0.7230010.3.1.4.8323329.27915.15178744...,NaN,78,457
8,1.2.276.0.7230010.3.1.4.8323329.4188.151787430...,"{""x"": 323.0, ""y"": 577.0, ""width"": 160.0, ""heig...",78,458
9,1.2.276.0.7230010.3.1.4.8323329.4188.151787430...,"{""x"": 695.0, ""y"": 575.0, ""width"": 162.0, ""heig...",78,458


## Server side 

### TODO: graphql & resolver 

- create ml model id per job, ml model id is unique, optionally allow user to specify ml model name 

- validate label scope, type 
- validate label annotation mode vs data field 
- return success/failure

**possible new ux feature**
- batch delete by ml model id/name 

## Client Side 

### TODO: Upload annotations

In [ ]:
# see implementation below. should use mdai.Client() to auth 
res = mdai_client.upload_anns(anns)

In [2]:
def upload_anns(anns): 
    # TODO: 
    # batch (implement as python generator)
    # call graphql query 
    # report progress 
    # report error 
    
    # TODO: need unique job id 
    # - ml model id stays same for this job id 
    # 
    
    # TODO: batch generator for anns 
    for batch in BatchGenerator(anns):
        query = '''
        mutation CreateDicomAnnotationBatch($input: CreateDicomAnnotationBatchInput!) {
          createDicomAnnotationBatch(input: $input) {
            id
          }
        }
        '''
        variables = {'input': batch}

        data = {'query': query, 'variables': variables}
        
        # TODO: 
        # - check to see if uploading is successful for current batch 
        # - (maybe) keep track of 

        headers={
            'Accept': 'application/json', 
            'Content-Type': 'application/json', 
            'Authorization': 'Bearer {}'.format(auth_token)
        }
        r = session.post('https://public.md.ai/api/graphql', headers=headers, json=data)

        if r.status_code != 200:
            print('error creating annotation', r.status_code)
            print(r)
            break
        else:
            body = r.json()
            if 'errors' in body:
                print('error creating annotation', body['errors'])
    

In [3]:
## TODO: implement batch generator for anns 
def BatchGenerator(anns):
    # TODO: batch anns
    return batch_anns